# Using mask to preprocess fMRI signals

In [3]:
import numpy as np
import nilearn
from nilearn.input_data import NiftiMasker
from nilearn.masking import compute_epi_mask
from nilearn.image import resample_to_img, load_img
import nibabel as nib
import matplotlib.pyplot as plt

import os
import random
import math
import time

### mask

mask是一个全部由0和1组成的nii文件。具体的含义是对应的图像位置是0就丢弃，1就保留。因为原始的nii图像非常的大，体素点很多，涉及到的计算量会非常大，mask丢弃了很多无关紧要的部分，只保留了大脑图像，有利于只关注重点区域，减少计算量。
- 注意！之所以能用mask对fMRI图像进行处理，这是因为fMRI图像事先进行了预处理，尤其是对原始的fMRI图像进行了配准，将其配准到了标准空间。如果没有经过预处理的图像，是无法直接使用mask的方法的。

In [6]:
mask_img = "../stAAE/data/ADHD200_mask_152_4mm.nii.gz"
masker = NiftiMasker(mask_img=mask_img, 
                     standardize=True,
                     detrend=1,
                     smoothing_fwhm=6.)
masker.fit()

NiftiMasker(detrend=1, mask_img='../stAAE/data/ADHD200_mask_152_4mm.nii.gz',
            smoothing_fwhm=6.0, standardize=True)

In [10]:
nii_img = "../stAAE/data/adhd/data/0010042/0010042_rest_tshift_RPI_voreg_mni.nii.gz"
img = load_img(nii_img)
img.shape

(61, 73, 61, 176)

In [11]:
mask = load_img(mask_img)
mask.shape

(49, 58, 47)

In [14]:
mask.get_fdata().sum()

28546.0

mask里面有28546个体素值是1，意思是保留这28546个体素的位置的值。将fMRI的图像通过mask转换成1D的fMRI信号就是只保留了28546个值了，fMRI图像一共有176个时间步，所以转换后的size是$(176,28546)$

In [15]:
fmri_masked = masker.transform(img)
fmri_masked.shape

(176, 28546)

通过masker还可以将1D的fMRI信号还原成3D的图像，还原后的图像和mask是同样大小的。

In [16]:
img = masker.inverse_transform(fmri_masked)
img.shape

(49, 58, 47, 176)

如果要应用深度学习做fMRI信号处理可以考虑使用3D信号作为输入，也可以考虑使用1D信号作为输入，以上是将nii图像处理为1D或者3D信号的方法。